In [2]:
from collections import deque, defaultdict
import itertools
import math
from heapq import heappush, heappop
import numpy as np
import sympy
import networkx as nx

In [15]:
# Day 25

# with open("day25/test.txt") as f:
with open("day25/input.txt") as f:
  adjlist = []
  for line in f.read().splitlines():
    adjlist.append(line.replace(":", ""))

  G = nx.parse_adjlist(adjlist)
  # nx.nx_pydot.write_dot(G, "graph.dot")
  # cuts = nx.minimum_edge_cut(G)
  _, (p1, p2) = nx.stoer_wagner(G)
  print(len(p1) * len(p2))

583632


In [87]:
# Day 24

class Hail:
  def __init__(self, pos, vel):
    self.x = pos[0]
    self.y = pos[1]
    self.z = pos[2]

    self.vx = vel[0]
    self.vy = vel[1]
    self.vz = vel[2]

  def intersect(self, other):
    A = np.array([[self.vx, -other.vx], [self.vy, -other.vy]])
    b = np.array([other.x - self.x, other.y - self.y])
    rank = np.linalg.matrix_rank(A)
    if rank == 1:
      return None
    t = np.linalg.solve(A, b)
    if len(t) != len(b):
      return None
    return t

# # with open("day24/test.txt") as f:
# with open("day24/input.txt") as f:
#   hail = []
#   for line in f.read().splitlines():
#     pos, vel = line.split(" @ ")
#     pos = list(map(int, pos.split(", ")))
#     vel = list(map(int, vel.split(", ")))
#     hail.append(Hail(pos, vel))

#   lo = 200000000000000
#   hi = 400000000000000
#   total = 0
#   for i, h1 in enumerate(hail):
#     for h2 in hail[i + 1:]:
#       cross = h1.intersect(h2)
#       if cross is not None and all(p >= 0 for p in cross):
#         t1, _ = cross
#         px, py = h1.x + t1 * h1.vx, h1.y + t1 * h1.vy
#         if lo <= px <= hi and lo <= py <= hi:
#           total += 1
#   print(total)

# part 2
# with open("day24/test.txt") as f:
with open("day24/input.txt") as f:
  hail = []
  for line in f.read().splitlines():
    pos, vel = line.split(" @ ")
    pos = list(map(int, pos.split(", ")))
    vel = list(map(int, vel.split(", ")))
    hail.append(Hail(pos, vel))

  eqs = []
  rx, ry, rz, vrx, vry, vrz = sympy.symbols("rx, ry, rz, vrx, vry, vrz")

  for i, h in enumerate(hail[:6]):
    x, y, z, vx, vy, vz = h.x, h.y, h.z, h.vx, h.vy, h.vz

    t = sympy.symbols(f"t{i}")

    eqs.append((x + vx * t) - (rx + vrx * t))
    eqs.append((y + vy * t) - (ry + vry * t))
    eqs.append((z + vz * t) - (rz + vrz * t))

  sol = sympy.solve(eqs)[0]
  print(sol[rx] + sol[ry] + sol[rz])




527310134398221


In [123]:
# Day 23

dirs = {
    "down": (1, 0),
    "up": (-1, 0),
    "left": (0, -1),
    "right": (0, 1),
}

slopes = {
    ">": "right",
    "<": "left",
    "^": "up",
    "v": "down",
}

def dfsLen(g, cur, dst, distance=0, seen=set()):
  if cur == dst:
    return distance

  best = 0
  for r, c, d in g[cur]:
    if (r, c) not in seen:
      seen = set(seen)
      seen.add(cur)
      best = max(best, dfsLen(g, (r, c), dst, distance + d, seen))

  return best


def buildGraph(grid, start, end):
  graph = defaultdict(set)

  junctions = set([start, end])
  for r, row in enumerate(grid):
    for c, tile in enumerate(row):
      if tile != "#":
        nbrs = 0
        for _, (dr, dc) in dirs.items():
          nr, nc = r + dr, c + dc
          if nr in range(len(grid)) and nc in range(len(grid[0])):
            if grid[nr][nc] != "#":
              nbrs += 1
        if nbrs > 2:
          junctions.add((r, c))

  for j in junctions:
    q = deque([(*j, 0)])
    seen = set([j])
    while q:
      r, c, dist = q.pop()

      if (r, c) in junctions and (r, c) != j:
        graph[j].add((r, c, dist))
        continue

      for dir, (dr, dc) in dirs.items():
          nr, nc = r + dr, c + dc
          if nr in range(len(grid)) and nc in range(len(grid[0])) and (nr, nc) not in seen:
            tile = grid[nr][nc]
            if tile != "#": #tile == "." or (tile in slopes and slopes[tile] == dir):
              q.append((nr, nc, dist + 1))
              seen.add((nr, nc))

  return graph

# with open("day23/test.txt") as f:
with open("day23/input.txt") as f:
  grid = f.read().splitlines()

  start = (0, grid[0].index("."))
  end = (len(grid) -1 , grid[len(grid) - 1].index("."))
  graph = buildGraph(grid, start, end)

  print(dfsLen(graph, start, end))



6298


In [ ]:
# Day 22

def pprint(bricks):
  maxX = max(b.maxX for b in bricks)
  maxY = max(b.maxY for b in bricks)
  maxZ = max(b.maxZ for b in bricks)

  print("-- x --")
  grid = [["." for _ in range(maxX + 1)] for _ in range(maxZ + 1)]
  for brick in bricks:
    for x in range(brick.minX, brick.maxX + 1):
      for z in range(brick.minZ, brick.maxZ + 1):
        grid[z][x] = brick.label
  for line in reversed(grid):
    print("".join(line))

  print("\n-- y --")
  grid = [["." for _ in range(maxY + 1)] for _ in range(maxZ + 1)]
  for brick in bricks:
    for y in range(brick.minY, brick.maxY + 1):
      for z in range(brick.minZ, brick.maxZ + 1):
        grid[z][y] = brick.label
  for line in reversed(grid):
    print("".join(line))


class Brick:
  def __init__(self, start, end, label):
    self.minX = start[0]
    self.minY = start[1]
    self.minZ = start[2]
    self.maxX = end[0]
    self.maxY = end[1]
    self.maxZ = end[2]
    self.label = label

  def __lt__(self, other):
    return self.minZ < other.minZ

  def __eq__(self, other):
    return self.minZ == other.minZ

  def intersect(self, other):
    return self.minX <= other.maxX and self.maxX >= other.minX and self.minY <= other.maxY and self.maxY >= other.minY

  def __repr__(self):
    return f"B:{self.label}"

# with open("day22/test.txt") as f:
with open("day22/input.txt") as f:
  bricks = []
  for i, line in enumerate(f.read().splitlines()):
    start, end = line.split("~")
    start, end = start.split(","), end.split(",")
    start, end = tuple(map(int, start)), tuple(map(int, end))
    bricks.append(Brick(start, end, str(i)))

  bricks.sort()
  for i, brick in enumerate(bricks):
    posZ = 1
    for below in bricks[:i]:
      if brick.intersect(below):
        posZ = max(posZ, below.maxZ + 1)
    height = brick.maxZ - brick.minZ
    brick.minZ = posZ
    brick.maxZ = posZ + height

  bricks.sort()
  supports = {}
  supportedBy = {}

  for i, brick in enumerate(bricks):
    supports[brick.label] = set()
    supportedBy[brick.label] = set()
    possible = []
    highestZ = 1
    for below in bricks[:i]:
      if brick.intersect(below):
        if below.maxZ >= highestZ:
          highestZ = below.maxZ
          possible.append(below)
    possible = filter(lambda b: b.maxZ == highestZ, possible)

    for below in possible:
      supports[below.label].add(brick.label)
      supportedBy[brick.label].add(below.label)

  total = 0
  for brick, children in supports.items():
    safe = True
    for child in children:
      if supportedBy[child] == set([brick]):
        safe = False
        break
    if safe:
      total += 1

  print(total)

  total = 0
  for brick in supports:
    q = deque([brick])
    broken = set([brick])
    while q:
      next = q.popleft()
      for child in supports[next]:
        if not child in broken and len(supportedBy[child].difference(broken)) == 0:
          broken.add(child)
          q.append(child)
    total += len(broken) - 1
  print(total)


In [ ]:
# Day 21

def pprint(grid, gardens):
  grid = list(map(list, grid))
  for r in range(len(grid)):
    for c in range(len(grid[r])):
      if (r,c) in gardens:
        grid[r][c] = "O"
  for line in grid:
    print("".join(line))

# with open("day21/test.txt") as f:
with open("day21/input.txt") as f:
  grid = f.read().splitlines()
  (startingPos, ) = [(r, c) for r in range(len(grid)) for c in range(len(grid[0])) if grid[r][c] == "S"]

  # print(len(grid), len(grid[0])) # 131 x 131
  intervals = [64]
  stepResults = []

  for i, steps in enumerate(intervals, start = 1):
    print("-"*10, f" Run {i} ", "-" * 50)

    gardens = set()
    seen = set()
    q = deque([(*startingPos, steps)])

    while q:
      r , c, steps = q.popleft()

      if steps % 2 == 0:
        gardens.add((r, c))
      if steps <= 0:
        continue

      for dr, dc in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
        nr, nc = r + dr, c + dc
        wr = nr % len(grid)
        wc = nc % len(grid[wr])
        if grid[wr][wc] != "#" and (nr, nc) not in seen:
          seen.add((nr, nc))
          q.append((nr, nc, steps - 1))
    stepResults.append(len(gardens))

  print(intervals)
  print(stepResults)

----------  Run 1  --------------------------------------------------
[64]
[3776]


In [ ]:
# [65, 196, 327, 458]
# [3884, 34564, 95816, 187640]

In [ ]:
# Day 20

# with open("day20/test.txt") as f:
with open("day20/input.txt") as f:
  nodes = {}
  for line in f.read().splitlines():
    name, children = line.split(" -> ")
    children = children.split(", ")
    if name[0] == "%":
      category = "flip"
      name = name[1:]
      state = 0
    elif name[0] == "&":
      category = "con"
      name = name[1:]
      state = {}
    else:
      category = "broad"
      state = None
    nodes[name] = {"category": category, "children": children, "state": state}

  for node, data in nodes.items():
    for child in data["children"]:
      if child in nodes and nodes[child]["category"] == "con":
        nodes[child]["state"][node] = 0

  # low, high
  pulses = [0, 0]
  for _ in range(1000):
    q = deque()
    q.append((0, "broadcaster", "button"))
    while q:
      pulse, name, source = q.popleft()
      pulses[pulse] += 1

      if name not in nodes:
        continue

      node = nodes[name]
      match node["category"]:
        case "broad":
          pulseToSend = pulse
        case "flip":
          if pulse:
            continue
          node["state"] = (1 + node["state"]) % 2
          pulseToSend = node["state"]
        case "con":
          node["state"][source] = pulse
          pulseToSend = 0 if all(s == 1 for s in node["state"].values()) else 1
      for child in node["children"]:
          q.append((pulseToSend, child, name))

  print(pulses[0] * pulses[1])

# Part 2
with open("day20/input.txt") as f:
  nodes = {}
  for line in f.read().splitlines():
    name, children = line.split(" -> ")
    children = children.split(", ")
    if name[0] == "%":
      category = "flip"
      name = name[1:]
      state = 0
    elif name[0] == "&":
      category = "con"
      name = name[1:]
      state = {}
    else:
      category = "broad"
      state = None
    nodes[name] = {"category": category, "children": children, "state": state}

  for node, data in nodes.items():
    for child in data["children"]:
      if child in nodes and nodes[child]["category"] == "con":
        nodes[child]["state"][node] = 0

  targetSource = "vd"
  penultimates = ["rd", "bt", "fv", "pr"]
  cycles = {}
  seen = {}
  presses = 0
  while True:
    presses += 1
    q = deque()
    q.append((0, "broadcaster", "button"))
    while q:
      pulse, name, source = q.popleft()

      if name not in nodes:
        continue

      if len(cycles) == 4:
        break


      if name == targetSource and source in penultimates and pulse:
        if source not in seen:
          seen[source] = presses
        else:
          cycles[source] = presses - seen[source]


      node = nodes[name]
      match node["category"]:
        case "broad":
          pulseToSend = pulse
        case "flip":
          if pulse:
            continue
          node["state"] = (1 + node["state"]) % 2
          pulseToSend = node["state"]
        case "con":
          node["state"][source] = pulse
          pulseToSend = 0 if all(s == 1 for s in node["state"].values()) else 1
      for child in node["children"]:
          q.append((pulseToSend, child, name))
    else:
      continue
    break

  print(math.lcm(*cycles.values()))


788848550
228300182686739


In [ ]:
# Day 19

# with open("day19/test.txt") as f:
with open("day19/input.txt") as f:
  rawWorkflows, rawParts = f.read().split("\n\n")

  workflows = {}
  for rw in rawWorkflows.split("\n"):
    name, rest = rw.split("{")
    steps = rest[:-1].split(",")
    instructions = []
    terminate = steps[-1]
    for step in steps[:-1]:
      condition, next = step.split(":")
      label, operator = condition[:2]
      qty = int(condition[2:])

      instructions.append((label, operator, qty, next))
    workflows[name] = {"instructions": instructions, "terminate": terminate}

  q = deque([])
  for p in rawParts.split("\n"):
    ranks = p[1:-1].split(",")
    part = {}
    for rank in ranks:
      cat, qty = rank.split("=")
      part[cat] = int(qty)
    q.append({"part": part, "next": "in"})

  total = 0
  while q:
    todo = q.popleft()
    wf = workflows[todo["next"]]
    part = todo["part"]

    goNext = False
    for label, operator, qty, next in wf["instructions"]:
      if operator == "<" and part[label] < qty:
        goNext = next
      elif operator == ">" and part[label] > qty:
        goNext = next
      if goNext:
        break

    if goNext:
      if goNext == "A":
        total += sum(part.values())
      elif goNext != "R":
        q.append({"part": part, "next": goNext})
    else:
      if wf["terminate"] == "A":
        total += sum(part.values())
      elif wf["terminate"] != "R":
        q.append({"part": part, "next": wf["terminate"]})


  print(total)


## Part 2
# with open("day19/test.txt") as f:
with open("day19/input.txt") as f:
  rawWorkflows, _ = f.read().split("\n\n")

  workflows = {}
  for rw in rawWorkflows.split("\n"):
    name, rest = rw.split("{")
    steps = rest[:-1].split(",")
    instructions = []
    terminate = steps[-1]
    for step in steps[:-1]:
      condition, next = step.split(":")
      label, operator = condition[:2]
      qty = int(condition[2:])

      instructions.append((label, operator, qty, next))
    workflows[name] = {"instructions": instructions, "terminate": terminate}

  possibleRanges = []

  def processWorkflows(next, currentRange):
    if next == "A":
        possibleRanges.append(currentRange)
    elif next != "R":
      wf = workflows[next]

      newRange = currentRange.copy()

      for label, operator, qty, nextWf in wf["instructions"]:
        min, max = newRange[label]
        if operator == "<":
          newRange[label] = (min, qty - 1)
          processWorkflows(nextWf, newRange)
          newRange = newRange.copy()
          newRange[label] = (qty, max)
        else:
          newRange[label] = (qty + 1, max)
          processWorkflows(nextWf, newRange)
          newRange = newRange.copy()
          newRange[label] = (min, qty)

      processWorkflows(wf["terminate"], newRange)

  startingRange = {
      "x": (1, 4000),
      "m": (1, 4000),
      "a": (1, 4000),
      "s": (1, 4000)
  }
  processWorkflows("in", startingRange)

  combinations = 0
  for r in possibleRanges:
    total = 1
    for min_, max_ in r.values():
      total *= (max_ - min_ + 1)
    combinations += total

  print(combinations)

In [ ]:
# Day 18

# with open("day18/test.txt") as f:
with open("day18/input.txt") as f:
  vertices = [(0, 0)]
  boundary = 0
  for line in f.read().splitlines():
    _, _, hex = line.split()
    hex = hex[2:-1]

    dir = {"0": "R", "1": "D", "2": "L", "3": "U"}[hex[-1]]
    qty = int(hex[:-1], 16)

    boundary += qty
    prevX, prevY = vertices[-1]
    match dir:
      case "R":
        nextX, nextY = prevX + qty, prevY
      case "L":
        nextX, nextY = prevX - qty, prevY
      case "D":
        nextX, nextY = prevX, prevY + qty
      case "U":
        nextX, nextY = prevX, prevY - qty
    vertices.append((nextX, nextY))

  # shoelace
  area = 0
  for i in range(len(vertices)):
    j = (i + 1) % len(vertices)
    area += vertices[i][0] * (vertices[j][1] - vertices[i - 1][1])
  area = abs(area) / 2

  # pick A = I + (b/2) - 1
  interior = area - (boundary / 2) + 1

  print(interior + boundary)


66296566363189.0


In [ ]:
# day 17

dirMap = {
    "left" : (0, -1),
    "right" : (0, 1),
    "up" : (-1, 0),
    "down" : (1, 0),
}

# with open("day17/test.txt") as f:
with open("day17/input.txt") as f:
   grid = f.read().splitlines()


   # dijkstra
   q = [(0, 0, 0, 0, None)]
   seen = set()
   while q:
    next = heappop(q)
    heat, r, c, sameDir, currDir = next

    if (r, c) == (len(grid) - 1, len(grid[0]) - 1) and sameDir >= 4:
      print(heat)
      break

    if next[1:] in seen:
      continue

    seen.add(next[1:])

    for dir, (dr, dc) in dirMap.items():
      nr, nc = r + dr, c + dc
      if currDir and (-1 * dr, -1 * dc) == dirMap[currDir]:
        continue # No reverse

      if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]):
        nh = int(grid[nr][nc])
        if currDir == dir:
          if sameDir < 10:
            heappush(q, (heat + nh, nr, nc, sameDir + 1, dir))
        else:
          if sameDir > 3 or currDir is None:
            heappush(q, (heat + nh, nr, nc, 1, dir))



1017


In [ ]:
# Day 16

dirMap = {
    "left" : (0, -1),
    "right" : (0, 1),
    "up" : (-1, 0),
    "down" : (1, 0),
}

# with open("day16/test.txt") as f:
with open("day16/input.txt") as f:
  grid = f.read().splitlines()


  starts = []
  for i in range(len(grid)):
    starts.append((i, 0, "left", "right"))
    starts.append((i, len(grid) - 1, "right", "left"))
  for i in range(len(grid[0])):
    starts.append((0, i, "up", "down"))
    starts.append((len(grid[0]) - 1, i, "down", "up"))

  energizedOpts = []
  for start in starts:
    seen = set()
    energized = set()
    q = deque([start])
    while q:
      next = q.popleft()

      if next in seen:
        continue

      r, c, source, move = next

      if not (0 <= r < len(grid) and 0 <= c < len(grid[r])):
        continue

      energized.add((r, c))
      seen.add(next)
      tile = grid[r][c]
      dr, dc = dirMap[move]

      match tile:
        case ".":
          q.append((r + dr, c + dc, source, move))
        case "|":
          if move in ["up", "down"]:
            q.append((r + dr, c + dc, source, move))
          else:
            q.append((r + 1, c, "up", "down"))
            q.append((r - 1, c, "down", "up"))
        case "-":
          if move in ["left", "right"]:
            q.append((r + dr, c + dc, source, move))
          else:
            q.append((r, c + 1, "left", "right"))
            q.append((r, c - 1, "right", "left"))
        case "/":
          if move == "up":
            q.append((r, c + 1, "left", "right"))
          elif move == "down":
            q.append((r, c - 1, "right", "left"))
          elif move == "right":
            q.append((r - 1, c, "down", "up"))
          else:
            q.append((r + 1, c, "up", "down"))
        case "\\":
          if move == "up":
            q.append((r, c - 1, "right", "left"))
          elif move == "down":
            q.append((r, c + 1, "left", "right"))
          elif move == "right":
            q.append((r + 1, c, "up", "down"))
          else:
            q.append((r - 1, c, "down", "up"))

    energizedOpts.append(len(energized))

  print(max(energizedOpts))


7759


In [ ]:
# Day 15

def aocHash(x):
  total = 0
  for ch in x:
    total += ord(ch)
    total *= 17
    total %= 256
  return total

# with open("day15/test.txt") as f:
with open("day15/input.txt") as f:
  steps = f.read().split(",")
  total = 0
  for step in steps:
    total += aocHash(step)
  print(total)

# Part 2
# with open("day15/test.txt") as f:
with open("day15/input.txt") as f:
  steps = f.read().split(",")
  for i, step in enumerate(steps):
    if "=" in step:
      label, lens = step.split("=")
    else:
      label = step[:-1]
      lens = None
    steps[i] = (label, lens)

  boxes = [[] for _ in range(256)]
  lenses = {}
  for label, lens in steps:
    boxIdx = aocHash(label)
    if lens:
      if label not in boxes[boxIdx]:
        boxes[boxIdx].append(label)
      lenses[label] = int(lens)
    else:
      if label in boxes[boxIdx]:
        boxes[boxIdx].remove(label)

  total = 0
  for i, box in enumerate(boxes, start = 1):
    for j, lens in enumerate(box, start = 1):
      total += i * j * lenses[lens]

  print(total)

509784
230197


In [ ]:
# Day 14

def pprint(grid):
  for line in grid:
    print("".join(line))

def stringify(grid):
  return "".join("".join(line) for line in grid)

def calcWeight(grid):
  total = 0
  for r, row in enumerate(grid):
    total += (len(grid) - r) * row.count("O")
  return total

def cycle(grid):
  for _ in range(4):
    grid = tiltNorth(grid)
    grid = rotateCW(grid)
  return grid

def rotateCW(grid):
  grid = list(map(list, zip(*grid)))
  grid = list(map(lambda x: x[::-1], grid))
  return grid

def tiltNorth(grid):
  colGrid = list(map(list, zip(*grid)))
  for c in range(len(colGrid)):
    nextPos = 0
    for r in range(len(colGrid[c])):
      if colGrid[c][r] == "#":
        nextPos = r + 1
      elif colGrid[c][r] == "O":
        colGrid[c][nextPos], colGrid[c][r] = colGrid[c][r], colGrid[c][nextPos]
        nextPos += 1
  return list(map(list, zip(*colGrid)))

with open("day14/test.txt") as f:
# with open("day14/input.txt") as f:
  grid = list(map(list, f.read().splitlines()))
  cache = {}
  iters = 1_000_000_000
  for i in range(iters):
    grid = cycle(grid)
    flat = stringify(grid)
    if flat in cache:
      idx = cache[flat]
      cycles = i - idx
      extra = (iters - idx) % cycles
      for _ in range(extra - 1):
        grid = cycle(grid)
      break
    cache[flat] = i
  print(calcWeight(grid))



102055


In [ ]:
# Day 13

def findMirror(grid):
  for i in range(1, len(grid)):
    above = grid[:i]
    below = grid[i:]
    diffCount = 0
    for r1, r2 in zip(reversed(above), below):
      diffs = sum(a != b for a, b in zip(r1, r2))
      if diffs > 1:
        break
      diffCount += diffs
    else:
      if diffCount == 1:
        return i
  return 0

# with open("day13/test.txt") as f:
with open("day13/input.txt") as f:
  grids = f.read().split("\n\n")
  total = 0
  for grid in grids:
    grid = grid.splitlines()
    colGrid = list(zip(*grid))
    total += 100 * findMirror(grid) + findMirror(colGrid)
  print(total)


35915


In [ ]:
# Day 12

def countCombos(springs, groups, seen = {}):
  if len(springs) == 0:
    return len(groups) == 0

  if len(groups) == 0:
    return "#" not in springs

  if (springs, groups) in seen:
    return seen[(springs, groups)]

  count = 0
  if springs[0] in ".?":
    count += countCombos(springs[1:], groups)
  if springs[0] in "#?":
    ng = groups[0]
    if len(springs) >= ng and "." not in springs[:ng] and (len(springs) == ng or springs[ng] != "#"):
      count += countCombos(springs[ng + 1:], groups[1:])

  seen[(springs, groups)] = count
  return count

# with open("day12/test.txt") as f:
with open("day12/input.txt") as f:
  total = 0
  for line in f.readlines():
    springs, groups = line.split()
    groups = tuple(list(map(int, groups.split(",")))*5)

    springs = "?".join([springs] * 5)

    total += countCombos(springs, groups)
  print(total)


200097286528151


In [ ]:
# Day 11

distance_mod = 1_000_000 - 1

# with open("day11/test.txt") as f:
with open("day11/input.txt") as f:
  grid = f.read().splitlines()
  colGrid = list(zip(*grid))

  emptyRows = [i for i in range(len(grid)) if all(col == "." for col in grid[i])]
  emptyCols = [i for i in range(len(colGrid)) if all(col == "." for col in colGrid[i])]

  galaxies = []
  for r, row in enumerate(grid):
    for c, col in enumerate(row):
      if col == "#":
        galaxies.append((r, c))

  pairs = []
  for i in range(len(galaxies)):
    for j in range(i + 1, len(galaxies)):
      pairs.append((galaxies[i], galaxies[j], i + 1, j + 1))

  total = 0
  for g1, g2, i , j in pairs:
    y1, x1 = g1
    y2, x2 = g2
    distance = abs(x2 - x1) + abs(y2 - y1)
    for r in emptyRows:
      if y1 < r < y2:
        distance += distance_mod
    for c in emptyCols:
      if min(x1, x2) < c < max(x1, x2):
        distance += distance_mod
    total += distance
  print(total)

630728425490


In [ ]:
# Day 10

dirs = {
    "N": (-1, 0),
    "S": (1, 0),
    "E": (0, 1),
    "W": (0, -1)
}

possiblePipes = {
    "N": "|7F",
    "S": "|JL",
    "E": "-J7",
    "W": "-LF",
}

pipeDirs = {
    "|": "NS",
    "-": "EW",
    "L": "NE",
    "F": "SE",
    "J": "NW",
    "7": "SW",
    "S": "NESW"
}

def replaceStart(grid, start):
  candidates = set(pipeDirs.keys())
  r, c = start
  for dir, (dr, dc) in dirs.items():
    nr, nc = r + dr, c + dc
    if 0 <= nr < len(grid) and 0 <= nc < len(grid[nr]):
      if grid[nr][nc] in possiblePipes[dir]:
        candidates = candidates.difference(possiblePipes[dir])
  grid[r] = grid[r].replace("S", candidates.pop())

# with open("day10/test.txt") as f:
with open("day10/input.txt") as f:
  grid = f.read().splitlines()
  startingPos = None

  for r, row in enumerate(grid):
    for c, col in enumerate(row):
      if col == "S":
        startingPos = (r, c)
        break

  replaceStart(grid, startingPos)

  steps = 0
  previousPos = None
  currentPos = startingPos
  while True:
    if currentPos == startingPos and steps > 0:
      break

    steps += 1
    r, c = currentPos
    currentPipe = grid[r][c]
    nextPos = None
    for dir in pipeDirs[currentPipe]:
      dr, dc = dirs[dir]
      nr, nc = r + dr, c + dc
      if 0 <= nr < len(grid) and 0 <= nc < len(grid[nr]):
        if (nr, nc) != previousPos:
          nextPos = (nr, nc)
          break

    previousPos = currentPos
    currentPos = nextPos

  print(steps / 2)

# Part 2
# with open("day10/test.txt") as f:
with open("day10/input.txt") as f:
  grid = f.read().splitlines()
  startingPos = None

  for r, row in enumerate(grid):
    for c, col in enumerate(row):
      if col == "S":
        startingPos = (r, c)
        break

  replaceStart(grid, startingPos)

  previousPos = None
  currentPos = startingPos
  steps = 0
  vertices = []
  while True:
    if currentPos == startingPos and steps > 0:
      break

    steps += 1
    started = True
    r, c = currentPos
    currentPipe = grid[r][c]
    nextPos = None

    if currentPipe in "FJ7L":
      vertices.append(currentPos)

    for dir in pipeDirs[currentPipe]:
      dr, dc = dirs[dir]
      nr, nc = r + dr, c + dc
      if 0 <= nr < len(grid) and 0 <= nc < len(grid[nr]):
        if (nr, nc) != previousPos:
          nextPos = (nr, nc)
          break

    previousPos = currentPos
    currentPos = nextPos

  # shoelace
  area = 0
  for i in range(len(vertices)):
    j = (i + 1) % len(vertices)
    area += vertices[i][1] * (vertices[j][0] - vertices[i - 1][0])
  area = abs(area) / 2

  # pick
  interior = area - (steps / 2) + 1

  print(interior)


7086.0
317.0


In [ ]:
# Day 9

def extrapolate(nums):
  if all(num == 0 for num in nums):
    return 0

  diffs = [nums[i+1] - nums[i] for i in range(len(nums) - 1)]
  return nums[-1] + extrapolate(diffs)

# with open("day9/test.txt") as f:
with open("day9/input.txt") as f:
  total = 0
  for line in f.readlines():
    nums = list(map(int, line.split()))
    nums.reverse()
    total += extrapolate(nums)
  print(total)

995


In [ ]:
# Day 8

# Part 1
# with open("day8/test.txt") as f:
with open("day8/input.txt") as f:
  directions, rest = f.read().split("\n\n")
  network = {}
  for line in rest.split("\n"):
    node, children = line.split(" = ")
    left, right = children[1:-1].split(", ")
    network[node] = (left, right)

  currentNode = "AAA"
  dirIdx = 0
  steps = 0
  while True:
    steps += 1
    nextDir = 0 if directions[dirIdx] == "L" else 1
    nextNode = network[currentNode][nextDir]
    if nextNode == "ZZZ":
      break
    currentNode = nextNode
    dirIdx = (dirIdx + 1) % len(directions)

  print(steps)

# Part 2

# with open("day8/test.txt") as f:
with open("day8/input.txt") as f:
  directions, rest = f.read().split("\n\n")
  network = {}
  for line in rest.split("\n"):
    node, children = line.split(" = ")
    left, right = children[1:-1].split(", ")
    network[node] = (left, right)

  starts = list(filter(lambda x: x[-1] == "A", network))
  cycles = []

  for start in starts:
    steps = 0
    dirIdx = 0
    currentNode = start
    foundZ = False
    while True:
      steps += 1
      nextDir = 0 if directions[dirIdx] == "L" else 1
      nextNode = network[currentNode][nextDir]
      if nextNode[-1] == "Z":
        if not foundZ:
            foundZ = True
            steps = 0
        else:
            cycles.append(steps)
            break
      currentNode = nextNode
      dirIdx = (dirIdx + 1) % len(directions)

  print(math.lcm(*cycles))

19631
21003205388413


In [ ]:
# Day 7

strength = ["J", "2", "3", "4", "5", "6", "7", "8", "9", "T", "Q", "K", "A"]

# with open("day7/test.txt") as f:
with open("day7/input.txt") as f:
  hands = []
  for line in f.readlines():
    hand, bid = line.split()
    nonJoker = list(filter(lambda x: x != "J", hand))
    jokerCount = len(hand) - len(nonJoker)
    rankedHand = sorted(list(len(list(g)) for _, g in itertools.groupby(sorted(nonJoker))))
    if not len(rankedHand):
      rankedHand.append(0)
    rankedHand[-1] += jokerCount
    rankedHand = "".join(map(str, rankedHand))
    valueHand = tuple(strength.index(card) for card in hand)
    match rankedHand:
      case "5":
        rank = 7
      case "14":
        rank = 6
      case "23":
        rank = 5
      case "113":
        rank = 4
      case "122":
        rank = 3
      case "1112":
        rank = 2
      case "11111":
        rank = 1
    hands.append((rank, valueHand, bid))

  total = 0
  for i, (_, _, bid) in enumerate(sorted(hands), start = 1):
    total += i * int(bid)

  print(total)



253499763


In [ ]:
# Day 6

# with open("day6/test.txt") as f:
with open("day6/input.txt") as f:
  times, distances = f.read().split("\n")
  time = int(times.split(":")[1].replace(" ", ""))
  distance = int(distances.split(":")[1].replace(" ", ""))

  total = 0
  for i in range(1, time + 1):
    move = i * (time - i)
    if move > distance:
      total += 1

  print(total)

23654842


In [ ]:
# Day 5

# Part 1
# with open("day5/test.txt") as f:
with open("day5/input.txt") as f:
  seedLine, *rawMaps = f.read().split("\n\n")
  seeds = list(map(int, seedLine.split(": ")[1].split()))

  maps = []
  for m in rawMaps:
    rows = []
    m = m.split("\n")[1:]
    for r in m:
      dest, source, rge = list(map(int, r.split()))
      rows.append((source, source + rge - 1, dest, dest + rge - 1))
    maps.append(rows)

  for m in maps:
    newSeeds = []
    for seed in seeds:
      for ss, se, ds, de in m:
        if ss <= seed <= se:
          newSeeds.append(ds + (seed - ss))
          break
      else:
        newSeeds.append(seed)
    seeds = newSeeds

  print(min(seeds))

# Part 2
# with open("day5/test.txt") as f:
with open("day5/input.txt") as f:
  seedLine, *rawMaps = f.read().split("\n\n")
  seeds = list(map(int, seedLine.split(": ")[1].split()))
  seeds = [(seeds[i], seeds[i] + seeds[i + 1] - 1) for i in range(0, len(seeds), 2)]

  maps = []
  for m in rawMaps:
    rows = []
    m = m.split("\n")[1:]
    for r in m:
      dest, source, rge = map(int, r.split())
      rows.append((source, source + rge - 1, dest, dest + rge - 1))
    maps.append(rows)

  seeds = deque(seeds)
  for m in maps:
    newSeeds = []
    while seeds:
      seedStart, seedEnd = seeds.popleft()
      for ss, se, ds, de in m:
        if ss <= seedStart and seedEnd <= se:
          newSeeds.append((ds + (seedStart - ss), ds + (seedEnd - ss)))
          break
        elif seedStart <= se and seedEnd > se:
          newSeeds.append((ds + (seedStart - ss), de))
          seeds.append((se + 1, seedEnd))
          break
        elif seedStart < ss and seedEnd >= ss:
          newSeeds.append((ds, ds + (seedEnd - ss)))
          seeds.append((seedStart, ss - 1))
          break
        elif seedStart < ss and seedEnd > se:
          newSeeds.append((ds, de))
          seeds.append((seedStart, ss - 1))
          seeds.append((se + 1, seedEnd))
          break
      else:
        newSeeds.append((seedStart, seedEnd))
    seeds = deque(newSeeds)

  print(min(seeds))

1181555926
(37806486, 68589802)


In [ ]:
# Day 4

#part 1
# with open("day4/test.txt") as f:
with open("day4/input.txt") as f:
  total = 0
  for line in f.readlines():
    points = 0

    winningNums, myNums = line.split(":")[1].split("|")
    winningSet = set()
    for num in winningNums.split():
      winningSet.add(num)
    for num in myNums.split():
      if num in winningSet:
        if points == 0:
          points += 1
        else:
          points *= 2

    total += points
  print(total)

#part 2
# with open("day4/test.txt") as f:
with open("day4/input.txt") as f:
  lines = f.readlines()
  cards = [1 for _ in range(len(lines))]
  for i, line in enumerate(lines):
    matches = 0
    winningNums, myNums = line.split(":")[1].split("|")
    winningSet = set()
    for num in winningNums.split():
      winningSet.add(num)
    for num in myNums.split():
      if num in winningSet:
        matches += 1

    for j in range(i + 1, i + matches + 1):
      if j < len(cards):
        cards[j] += cards[i]

  print(sum(cards))

17803
5554894


In [ ]:
# Day 3

adjacentDeltas = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

# with open("day3/test.txt") as f:
with open("day3/input.txt") as f:
  grid = f.read().splitlines()
  total = 0
  for r, row in enumerate(grid):
    for c, col in enumerate(row):
      if col != "*":
        continue

      numStart = set()

      for deltaR, deltaC in adjacentDeltas:
        pointR = r + deltaR
        pointC = c + deltaC
        if not (0 <= pointR < len(grid) and 0 <= pointC < len(grid[pointR])):
          continue
        if not grid[pointR][pointC].isdigit():
          continue
        while pointC >= 0 and grid[pointR][pointC].isdigit():
          pointC -= 1
        numStart.add((pointR, pointC + 1))

      if len(numStart) != 2:
        continue

      nums = []

      for r2, c2 in numStart:
        endC = c2
        while endC < len(grid[r2]) and grid[r2][endC].isdigit():
          endC += 1
        nums.append(int(grid[r2][c2:endC]))

      total += nums[0] * nums[1]

  print(total)

81709807


In [ ]:
# Day 2

# part 1
with open("day2/input.txt") as f:
  maxCubes = {
      "red": 12,
      "green": 13,
      "blue": 14
  }

  total = 0
  for i, line in enumerate(f.read().splitlines(), start = 1):
    _, rounds = line.split(": ")
    for round in rounds.split("; "):
      cubes = round.split(", ")
      for cube in cubes:
        count, color = cube.split()
        count = int(count)
        if maxCubes[color] < count:
          break
      else:
        continue
      break
    else:
      total += i
  print(total)

# part 2
with open("day2/input.txt") as f:
  total = 0
  for i, line in enumerate(f.read().splitlines(), start = 1):

    minCubes = {
      "red": 1,
      "green": 1,
      "blue": 1
    }

    _, rounds = line.split(": ")
    for round in rounds.split("; "):
      cubes = round.split(", ")
      for cube in cubes:
        count, color = cube.split()
        count = int(count)
        minCubes[color] = max(minCubes[color], count)

    total += minCubes["red"] * minCubes["green"] * minCubes["blue"]
  print(total)




2268
63542


In [ ]:
# Day 1

wordNums = {
  "one": 1,
  "two": 2,
  "three": 3,
  "four": 4,
  "five": 5,
  "six": 6,
  "seven": 7,
  "eight": 8,
  "nine": 9
}

with open("day1/input.txt") as f:
  total = 0
  for line in f.read().splitlines():
    firstNum = None
    lastNum = None
    for i, ch in enumerate(line):
      foundNum = None
      if ch.isdigit():
        foundNum = int(ch)
      else: # Part 2
        for word in wordNums:
          if i + len(word) <= len(line) and line[i : i + len(word)] == word:
            foundNum = wordNums[word]
            break

      if foundNum:
        if not firstNum:
          firstNum = foundNum
        lastNum = foundNum

    total += int(f"{firstNum}{lastNum}")

  print(total)

52834
